 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"></ul></div>

In [6]:
library(scran)
dir.create("results", showWarning=FALSE)


In [7]:
# Load the output of "preparedata.R".

In [8]:
load("logdataFandA_all.RData") 

In [9]:
colnames(logDataF3)[colnames(logDataF3)=="other"] <- "Unsorted"

In [10]:
colnames(logDataA3)[colnames(logDataA3)=="ERY"] <- "MEP"

In [11]:
raw.all <- cbind(logDataF3, logDataA3)

In [12]:
first.batch <- rep(c(TRUE, FALSE), c(ncol(logDataF3), ncol(logDataA3)))

In [13]:
# Adding colours.
#base.color <- "grey"
color.legendF <- c(MEP="orange", GMP="chartreuse4", CMP="magenta", 
                   HSPC="cyan", LTHSC="dodgerblue", MPP="blue", LMPP="light blue", Unsorted="grey")
colmatF <- col2rgb(color.legendF) 

colmatA <- colmatF + 100 # A lighter shade.
colmatA[colmatA > 255] <- 255


In [14]:
#colmatF<-colmatF+200
#colmatF[colmatF > 255] <- 255
color.legendA <- setNames(rgb(colmatA[1,], colmatA[2,], colmatA[3,], maxColorValue=255), names(color.legendF))
allcolors <- c(color.legendF[colnames(logDataF3)], color.legendA[colnames(logDataA3)])
batch<-c( rep(1,ncol(logDataF3)),rep(2,ncol(logDataA3)) )

In [15]:
# Only keeping common cell types for PCA.
celltypes <- c(colnames(logDataF3), colnames(logDataA3))
pca.retain <- celltypes %in% c("MEP", "GMP", "CMP") 

In [16]:
# Making a plotting function.
plotFUN <- function(fname, Y, subset=NULL, ..., xlab="tSNE 1",ylab="tSNE 2",main="") {
  if (is.null(subset)) {
    subset <- seq_len(nrow(Y))
  }
  png(fname,width=900,height=700)
  par(mfrow=c(1,1),mar=c(6,6,4,2),cex.axis=2,cex.main=3,cex.lab=2.5)
  plot(Y[,1], Y[,2], cex=2,
       pch=ifelse(first.batch, 21, 1)[subset], 
       col=ifelse(first.batch, "black", allcolors)[subset],
       bg=allcolors[subset], xlab=xlab, ylab=ylab, main=main) 
    dev.off()
}

In [17]:
batchcolor=c("lavender","lightcoral")
plotFUNb <- function(fname, Y, subset=NULL, ...) {
  if (is.null(subset)) {
    subset <- seq_len(nrow(Y))
  }
  png(fname,width=900,height=700)
  par(mfrow=c(1,1),mar=c(6,6,4,2),cex.axis=2,cex.main=3,cex.lab=2.5)
  plot(Y[,1], Y[,2], cex=2,
       pch=ifelse(first.batch, 21, 1)[subset], 
       col=ifelse(first.batch, "black", batchcolor[batch[subset]]),
       bg=batchcolor[batch[subset]], ...)#,  xlab="tSNE 1",ylab="tSNE 2")
  dev.off()
}


In [18]:
install.packages('Rtsne')

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [19]:
# No correction.

X.unc <- raw.all

In [20]:
t.unc <- t(X.unc)

In [21]:
## Generating a t-SNE plot.
#require()
set.seed(0)
all.dists.unc <- as.matrix(dist(t.unc))
tsne.unc <- Rtsne(all.dists.unc, is_distance=TRUE, perplexity = 90)
plotFUN("results/uncFA.png", tsne.unc$Y, main="Uncorrected",  xlab="tSNE 1",ylab="tSNE 2")
plotFUNb("results/uncFAb.png", tsne.unc$Y, main="Uncorrected",  xlab="tSNE 1",ylab="tSNE 2")

rm(all.dists.unc)
gc()

ERROR: Error in Rtsne(all.dists.unc, is_distance = TRUE, perplexity = 90): не могу найти функцию "Rtsne"


In [ ]:
# Generating a PCA plot.
pca.unc <- prcomp(t.unc[pca.retain,], rank=2)
pca.unc$x[ (pca.unc$x<(-0.08))]<- (-0.08)
plotFUN("results/pca_raw.png", pca.unc$x, subset=pca.retain, main="Uncorrected", ylim=c(-0.1, max(pca.unc$x[,2])),  xlab="PC 1",ylab="PC 2")

rm(t.unc)


In [6]:
mnnCorrect1 <- function(...)
{
    batches <- batches0 <- list(...)
    nbatches <- length(batches) 
    if (nbatches < 2L) { stop("at least two batches must be specified") }

    first <- batches[[1]]
    ref.nrow <- nrow(first)
    print(ref.nrow)
    ref.rownames <- rownames(first)  
    for (b in 2:nbatches) {
        current <- batches[[b]]
        if (!identical(nrow(current), ref.nrow)) {
            stop("number of rows is not the same across batches")
        } else if (!identical(rownames(current), ref.rownames)) {
            stop("row names are not the same across batches")
        }
    }

    # Subsetting to the desired subset of genes.
    if (!is.null(inquiry.genes)) {
        batches0 <- lapply(batches0, "[", i=inquiry.genes, , drop=FALSE) # Need the extra comma!
    }
    if (!is.null(hvg.genes)) { 
        batches <- lapply(batches, "[", i=hvg.genes, , drop=FALSE)
    }
    inquiry.genes <- .subset_to_index(inquiry.genes, first, byrow=TRUE)
    print(inquiry.genes)
}

In [8]:
functionBody(mnnCorrect)

{
    batches <- list(...)
    nbatches <- length(batches)
    if (nbatches < 2L) {
        stop("at least two batches must be specified")
    }
    prep.out <- prepare.input.data(batches, cos.norm.in = cos.norm.in, 
        cos.norm.out = cos.norm.out, subset.row = subset.row)
    in.batches <- prep.out$In
    out.batches <- prep.out$Out
    subset.row <- prep.out$Subset
    same.set <- prep.out$Same
    if (is.null(order)) {
        order <- seq_len(nbatches)
    }
    else {
        order <- as.integer(order)
        if (!identical(seq_len(nbatches), sort(order))) {
            stop(sprintf("'order' should contain values in 1:%i", 
                nbatches))
        }
    }
    ref <- order[1]
    ref.batch.in <- t(in.batches[[ref]])
    if (!same.set) {
        ref.batch.out <- t(out.batches[[ref]])
    }
    output <- vector("list", nbatches)
    output[[ref]] <- out.batches[[ref]]
    mnn.list <- vector("list", nbatches)
    mnn.list[[ref]] <- DataFrame(current.cell = integer(0),

In [5]:
dim(logDataA3)

[1] 3491 2729

In [ ]:
# Performing the correction with MNN (turned down the sigma to improve mixing).

mnn.out<-mnnCorrect(logDataF3, logDataA3,k=20, sigma=0.1,
                    cos.norm.in=TRUE, cos.norm.out=TRUE, var.adj=TRUE,compute.angle=TRUE)

X.mnn <- cbind(mnn.out$corrected[[1]], mnn.out$corrected[[2]])
t.mnn <- t(X.mnn)

In [ ]:
# plot histogram of angles between batch vectors and 2 svds of the reference batch.
png(file="results/angles.png",width=900,height=700)
par(mfrow=c(1,1),mar=c(6,6,4,2),cex.axis=2,cex.main=3,cex.lab=2.5)
hist(mnn.out$angles[[2]],xlab="Angle",ylab="Frequency",main="")
dev.off()

In [ ]:
# Generating a t-SNE plot.
set.seed(0)
all.dists.mnn <- as.matrix(dist(t.mnn))
tsne.mnn <- Rtsne(all.dists.mnn, is_distance=TRUE, perplexity = 90)
plotFUN("results/mnnFA.png", tsne.mnn$Y, main="MNN",  xlab="tSNE 1",ylab="tSNE 2")
plotFUNb("results/mnnFAb3.png", tsne.mnn$Y, main="MNN",  xlab="tSNE 1",ylab="tSNE 2")

In [ ]:
set.seed(0)
tsne.mnn2 <- Rtsne(t.mnn, perplexity = 90)
plotFUN("results/mnnFA_conventsne.png", tsne.mnn2$Y, main="MNN",  xlab="tSNE 1",ylab="tSNE 2")

In [ ]:
rm(all.dists.mnn)
gc()

In [ ]:
# Generating a PCA plot.
pca.mnn <- prcomp(t.mnn[pca.retain,], rank=2)
#pca.mnn$x[ (pca.mnn$x<(-0.08))]<- (-0.08)
plotFUN("results/pca_mnn.png", pca.mnn$x, subset=pca.retain, main="MNN", ylim=c(-0.08,0.05),  xlab="PC 1",ylab="PC 2")

In [ ]:
# Generating diffusion map plots.
library(destiny)
dm<-DiffusionMap(t.mnn,n_local = 150)
plotFUN("results/mnnFAdm12.png", dm@eigenvectors[,1:2], main="MNN",  xlab="DC 1",ylab="DC 2")
plotFUN("results/mnnFAdm23.png", cbind(dm@eigenvectors[,2],dm@eigenvectors[,3]), main="MNN",  xlab="DC 2",ylab="DC 4")
plotFUN("results/mnnFAdm13.png", cbind(dm@eigenvectors[,1],dm@eigenvectors[,3]), main="MNN",  xlab="DC 2",ylab="DC 4")

rm(t.mnn)
gc()
